In [13]:
import re
import os
import pandas as pd

In [14]:
from Karsimulator_Start_Genome import get_event_chr

In [15]:
files = []

for file in os.listdir('cluster_files/'):
	files.append(file)

files.sort()

In [16]:
data = []

for file in files:
	new_data = {}
	file_name = file.split('cluster')[0]
	cluster_number = file.split('cluster_')[1].replace('.txt', '')
	with open('cluster_files/' + file) as fp_read:
		line1 = fp_read.readline()
		matches = re.findall(r'<(.*?)>', line1)
		origins = eval(matches[1])
		new_data = {'file_name': file_name,
					'cluster': matches[0],
					'n_origin_chr': len(origins),
					'origin_chr': origins,
					'n_path_karsim': int(matches[2]),
					'n_path_omkar': int(matches[3])}
	alignment_file = file.split('.')[0] + '.alignment.txt'
	with open('alignment_files/' + alignment_file) as fp_read:
		line1 = fp_read.readline()
		line1 = line1.replace('\n', '').split(': ')[1]
		new_data['total_alignment_cost'] = int(line1)
		alignment_costs = []
		for line in fp_read:
			if line.startswith('alignment'):
				line = line.replace('\n', '').split('cost: ')[1]
				alignment_costs.append(int(line))
		new_data['alignment_costs'] = alignment_costs
	data.append(new_data)

In [17]:
df = pd.DataFrame(data)

In [22]:
(df['n_origin_chr'] >= 2).sum()

39

In [23]:
# get event_chr
karsim_file_prefix = '/media/zhaoyang-new/workspace/KarSim/KarComparator/new_data_files/KarSimulator/'
df['event_chr'] = df['file_name'].apply(lambda x: list(get_event_chr(karsim_file_prefix + x + '.kt.txt')))

In [24]:
df['event_chr'] = df['event_chr'].apply(lambda x: [entry[:-1] for entry in x])

In [25]:
df

,file_name,cluster,n_origin_chr,origin_chr,n_path_karsim,n_path_omkar,total_alignment_cost,alignment_costs,event_chr
0,23X_15q26_overgrowth_r1,0,1,[Chr1],2,2,64203,"[41160, 23043]","[Chr1, Chr2, Chr3, Chr5, Chr6, Chr8, Chr11, Ch..."
1,23X_15q26_overgrowth_r1,1,1,[Chr2],2,2,188445274,"[188232388, 212886]","[Chr1, Chr2, Chr3, Chr5, Chr6, Chr8, Chr11, Ch..."
2,23X_15q26_overgrowth_r1,10,1,[Chr11],2,2,22574,"[15518, 7056]","[Chr1, Chr2, Chr3, Chr5, Chr6, Chr8, Chr11, Ch..."
3,23X_15q26_overgrowth_r1,11,1,[Chr12],2,2,2696,"[1348, 1348]","[Chr1, Chr2, Chr3, Chr5, Chr6, Chr8, Chr11, Ch..."
4,23X_15q26_overgrowth_r1,12,1,[Chr13],2,1,183591686,"[96303080, 87288606]","[Chr1, Chr2, Chr3, Chr5, Chr6, Chr8, Chr11, Ch..."
...,...,...,...,...,...,...,...,...,...
603,23Y_WAGR_11p13_deletion_r2,5,1,[Chr7],2,2,1976,"[988, 988]","[Chr2, Chr3, Chr4, Chr10, Chr11, Chr15, Chr18,..."
604,23Y_WAGR_11p13_deletion_r2,6,1,[Chr8],2,2,5020,"[2510, 2510]","[Chr2, Chr3, Chr4, Chr10, Chr11, Chr15, Chr18,..."
605,23Y_WAGR_11p13_deletion_r2,7,1,[Chr9],2,2,504,"[252, 252]","[Chr2, Chr3, Chr4, Chr10, Chr11, Chr15, Chr18,..."
606,23Y_WAGR_11p13_deletion_r2,8,1,[Chr10],2,2,104410,"[102255, 2155]","[Chr2, Chr3, Chr4, Chr10, Chr11, Chr15, Chr18,..."


In [26]:
df['is_event_cluster'] = df.apply(lambda row: any(origin_chr in row['event_chr'] for origin_chr in row['origin_chr']), axis=1)

In [30]:
def count_overlaps(origin_chr, event_chr):
	count = 0
	for origin in origin_chr:
		if origin in event_chr:
			count += 1
	return count

# Apply the custom function to each row and create a new column
df['n_event_chr'] = df.apply(lambda row: count_overlaps(row['origin_chr'], row['event_chr']), axis=1)

In [31]:
df

,file_name,cluster,n_origin_chr,origin_chr,n_path_karsim,n_path_omkar,total_alignment_cost,alignment_costs,event_chr,is_event_cluster,n_event_chr
0,23X_15q26_overgrowth_r1,0,1,[Chr1],2,2,64203,"[41160, 23043]","[Chr1, Chr2, Chr3, Chr5, Chr6, Chr8, Chr11, Ch...",True,1
1,23X_15q26_overgrowth_r1,1,1,[Chr2],2,2,188445274,"[188232388, 212886]","[Chr1, Chr2, Chr3, Chr5, Chr6, Chr8, Chr11, Ch...",True,1
2,23X_15q26_overgrowth_r1,10,1,[Chr11],2,2,22574,"[15518, 7056]","[Chr1, Chr2, Chr3, Chr5, Chr6, Chr8, Chr11, Ch...",True,1
3,23X_15q26_overgrowth_r1,11,1,[Chr12],2,2,2696,"[1348, 1348]","[Chr1, Chr2, Chr3, Chr5, Chr6, Chr8, Chr11, Ch...",False,0
4,23X_15q26_overgrowth_r1,12,1,[Chr13],2,1,183591686,"[96303080, 87288606]","[Chr1, Chr2, Chr3, Chr5, Chr6, Chr8, Chr11, Ch...",True,1
...,...,...,...,...,...,...,...,...,...,...,...
603,23Y_WAGR_11p13_deletion_r2,5,1,[Chr7],2,2,1976,"[988, 988]","[Chr2, Chr3, Chr4, Chr10, Chr11, Chr15, Chr18,...",False,0
604,23Y_WAGR_11p13_deletion_r2,6,1,[Chr8],2,2,5020,"[2510, 2510]","[Chr2, Chr3, Chr4, Chr10, Chr11, Chr15, Chr18,...",False,0
605,23Y_WAGR_11p13_deletion_r2,7,1,[Chr9],2,2,504,"[252, 252]","[Chr2, Chr3, Chr4, Chr10, Chr11, Chr15, Chr18,...",False,0
606,23Y_WAGR_11p13_deletion_r2,8,1,[Chr10],2,2,104410,"[102255, 2155]","[Chr2, Chr3, Chr4, Chr10, Chr11, Chr15, Chr18,...",True,1


In [35]:
# see if the following statement is true: a cluster is only an event cluster IFF it only contains event_chr
event_cluster_with_non_event_chr = df['is_event_cluster'] & (df['n_origin_chr'] - df['n_event_chr'] > 0)
print(sum(event_cluster_with_non_event_chr))

0
